<a href="https://colab.research.google.com/github/ma2070-spec/Outamation_AI_Externship/blob/main/Project_3_%E2%80%93_PDF_Text_Extraction_%26_OCR_Comparison_(Tasks_1_%26_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TASK 1 Try to install a prebuilt wheel (fastest, least error-prone)
!pip -q install --only-binary=:all: "pymupdf<1.25" pandas || true

# Fallback if the previous line didn’t install fitz:
try:
    import fitz
    print("PyMuPDF is ready.")
except:
    print("Retrying with a pinned version (may build from source)…")
    !pip -q install "pymupdf==1.23.26" pandas
    import fitz
    print("PyMuPDF is ready (pinned).")



PyMuPDF is ready.


In [2]:
import os, json, pandas as pd
from google.colab import files

uploaded = files.upload()      # choose your Loan Worksheet PDF
pdf_path = list(uploaded.keys())[0]
print("Using file:", pdf_path)


Saving LenderFeesWorksheetNew (3).pdf to LenderFeesWorksheetNew (3) (8).pdf
Using file: LenderFeesWorksheetNew (3) (8).pdf


In [3]:
import fitz  # PyMuPDF

def extract_words_to_json(pdf_path, out_json='loan_worksheet_words.json', out_csv='loan_worksheet_words.csv'):
    doc = fitz.open(pdf_path)
    results, rows = [], []
    for p in range(len(doc)):
        page = doc[p]
        words = page.get_text("words")  # (x0,y0,x1,y1,word,block,line,word_no)
        words.sort(key=lambda w: (w[1], w[0]))
        for (x0,y0,x1,y1,txt,blk,ln,wn) in words:
            results.append({
                "page": p+1,
                "text": txt,
                "bbox": [float(x0), float(y0), float(x1), float(y1)],
                "block_no": int(blk), "line_no": int(ln), "word_no": int(wn)
            })
            rows.append([p+1, txt, x0, y0, x1, y1, blk, ln, wn])
    doc.close()

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    df = pd.DataFrame(rows, columns=["page","text","x0","y0","x1","y1","block_no","line_no","word_no"])
    df.to_csv(out_csv, index=False, encoding="utf-8")
    print(f"Saved {len(results)} items →", out_json)
    print("CSV preview saved →", out_csv)
    return df

df_words = extract_words_to_json(pdf_path)
df_words.head(15)



Saved 404 items → loan_worksheet_words.json
CSV preview saved → loan_worksheet_words.csv


,page,text,x0,y0,x1,y1,block_no,line_no,word_no
0,1,Your,36.000000,24.292059,60.943878,39.359196,0,0,0
1,1,actual,63.950756,24.292059,95.603142,39.359196,0,0,1
2,1,"rate,",98.610016,24.292059,121.743523,39.359196,0,0,2
3,1,"payment,",124.750397,24.292059,172.827774,39.359196,0,0,3
4,1,and,175.834671,24.292059,195.301544,39.359196,0,0,4
5,1,cost,198.308426,24.292059,220.823181,39.359196,0,0,5
6,1,could,223.830063,24.292059,253.036377,39.359196,0,0,6
7,1,be,256.043243,24.292059,268.818542,39.359196,0,0,7
8,1,higher.,271.825439,24.292059,308.342072,39.359196,0,0,8
9,1,Get,311.348969,24.292059,329.601288,39.359196,0,0,9


In [4]:
def to_lines(df: pd.DataFrame):
    for c in ["page","block_no","line_no","word_no","x0","y0","x1","y1"]:
        if c in df.columns: df[c] = pd.to_numeric(df[c], errors="coerce")
    lines = (
        df.sort_values(["page","y0","x0","word_no"])
          .groupby(["page","block_no","line_no"])
          .apply(lambda g: {
              "page": int(g["page"].iloc[0]),
              "block_no": int(g["block_no"].iloc[0]),
              "line_no": int(g["line_no"].iloc[0]),
              "text": " ".join(g["text"].tolist()),
              "bbox": [float(g["x0"].min()), float(g["y0"].min()),
                       float(g["x1"].max()), float(g["y1"].max())]
          }).tolist()
    )
    return lines

lines = to_lines(df_words)
import json
with open('loan_worksheet_lines.json','w',encoding='utf-8') as f:
    json.dump(lines, f, ensure_ascii=False, indent=2)
print("Saved line-level JSON → loan_worksheet_lines.json")
lines[:3]



Saved line-level JSON → loan_worksheet_lines.json


/tmp/ipython-input-1385719503.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: {


[{'page': 1,
  'block_no': 0,
  'line_no': 0,
  'text': 'Your actual rate, payment, and cost could be higher. Get an official Loan Estimate before choosing a loan.',
  'bbox': [36.0, 24.29205894470215, 587.3409423828125, 39.359195709228516]},
 {'page': 1,
  'block_no': 1,
  'line_no': 0,
  'text': 'Fee Details and Summary',
  'bbox': [230.39999389648438,
   56.20814895629883,
   376.87939453125,
   72.67018127441406]},
 {'page': 1,
  'block_no': 2,
  'line_no': 0,
  'text': 'Applicants:',
  'bbox': [39.599998474121094,
   77.31971740722656,
   77.63729858398438,
   88.00531768798828]}]

In [5]:


# TASK 2 --- 1. Upload PDF  ---
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print("Using file:", pdf_path)

Saving LenderFeesWorksheetNew (3).pdf to LenderFeesWorksheetNew (3) (9).pdf
Using file: LenderFeesWorksheetNew (3) (9).pdf


In [6]:
# --- 2. Install clean OCR environment ---
!pip -q install --upgrade pip
!apt-get -qq install -y tesseract-ocr poppler-utils
!pip -q install pytesseract pillow pdf2image pandas easyocr "paddleocr==2.7.0.3" "paddlepaddle==2.5.2"


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'PyMuPDF' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'PyMuPDF'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyMuPDF
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> PyMuPDF


In [7]:

# --- 3. Convert PDF to images ---
from pdf2image import convert_from_path
import os
os.makedirs("pages", exist_ok=True)
images = convert_from_path(pdf_path, dpi=300)
image_paths = []
for i, img in enumerate(images, start=1):
    out_path = f"pages/page_{i}.png"
    img.save(out_path)
    image_paths.append(out_path)
print("Pages saved:", image_paths)


Pages saved: ['pages/page_1.png']


In [8]:
# --- 4. Tesseract OCR ---
import pytesseract
tess_results = []
for img in image_paths:
    text = pytesseract.image_to_string(img)
    tess_results.append(text)
with open("tesseract_output.txt", "w") as f:
    f.write("\n\n".join(tess_results))


In [9]:
!pip -q uninstall -y easyocr torch torchvision torchaudio || true
!pip -q install --upgrade pip
# PyTorch CPU builds known to work well on Colab
!pip -q install torch==2.2.2+cpu torchvision==0.17.2+cpu torchaudio==2.2.2+cpu -f https://download.pytorch.org/whl/torch_stable.html
# EasyOCR pinned
!pip -q install easyocr==1.7.1


In [10]:
import torch

# Map unsigned dtypes that PyTorch doesn't expose to signed equivalents
_missing = []
for unsigned, signed in (("uint16","int16"), ("uint32","int32"), ("uint64","int64")):
    if not hasattr(torch, unsigned):
        setattr(torch, unsigned, getattr(torch, signed))
        _missing.append(unsigned)
print("Shimmed missing:", _missing)

# Force CPU for stability
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["USE_TORCH"] = "1"

import easyocr
reader_easy = easyocr.Reader(['en'], gpu=False)
print("EasyOCR ready.")



Shimmed missing: ['uint16', 'uint32', 'uint64']


EasyOCR ready.


In [11]:
easy_results = []
for img in image_paths:
    result = reader_easy.readtext(img, detail=0)  # just the strings
    easy_results.append("\n".join(result))
with open("easyocr_output.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(easy_results))
print("EasyOCR ✓ wrote easyocr_output.txt")


EasyOCR ✓ wrote easyocr_output.txt


In [12]:
# --- 5. EasyOCR ---
import easyocr
reader_easy = easyocr.Reader(['en'])
easy_results = []
for img in image_paths:
    result = reader_easy.readtext(img, detail=0)
    easy_results.append("\n".join(result))
with open("easyocr_output.txt", "w") as f:
    f.write("\n\n".join(easy_results))


In [13]:
# --- PaddleOCR (auto-install if runtime supports it; else skip cleanly) ---
import sys, subprocess, os, json
from PIL import Image

def try_install_paddle():
    # Paddle wheels are reliably available on Python <= 3.10.
    if sys.version_info[:2] <= (3,10):
        print("Trying to install PaddleOCR for Python", sys.version.split()[0])
        code1 = subprocess.run(
            ["python","-m","pip","-q","install","paddlepaddle==2.5.2"],
            capture_output=True, text=True
        )
        code2 = subprocess.run(
            ["python","-m","pip","-q","install","paddleocr==2.7.0.3"],
            capture_output=True, text=True
        )
        ok = (code1.returncode == 0) and (code2.returncode == 0)
        if not ok:
            print("Paddle install output (truncated):")
            print(code1.stderr[-500:])
            print(code2.stderr[-500:])
        return ok
    else:
        print("Skipping PaddleOCR install: Python", sys.version.split()[0], "is not eligible (>= 3.11).")
        return False

def run_paddle_ocr(image_paths):
    from paddleocr import PaddleOCR
    ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)
    os.makedirs("outputs/paddleocr", exist_ok=True)
    all_text = []
    for idx, img_path in enumerate(image_paths, start=1):
        res = ocr.ocr(img_path, cls=True)
        lines, boxes, page_json = [], [], []
        if res and len(res) > 0:
            for line in res[0]:
                bbox, (text, conf) = line
                lines.append(text)
                xs=[pt[0] for pt in bbox]; ys=[pt[1] for pt in bbox]
                boxes.append((int(min(xs)), int(min(ys)), int(max(xs)), int(max(ys))))
                page_json.append({"text": text, "conf": float(conf), "bbox": bbox})
        page_text = " ".join(lines)
        all_text.append(page_text)
        with open(f"outputs/paddleocr/page_{idx:02d}.txt","w",encoding="utf-8") as f: f.write(page_text)
        with open(f"outputs/paddleocr/page_{idx:02d}.json","w",encoding="utf-8") as f: json.dump(page_json, f, ensure_ascii=False, indent=2)
    with open("outputs/paddleocr/all_text.txt","w",encoding="utf-8") as f:
        f.write("\n\n".join(all_text))
    print("PaddleOCR ✓ → outputs/paddleocr/")

# Try to import; if missing, try to install; if still missing, skip
try:
    from paddleocr import PaddleOCR  # type: ignore
    HAVE_PADDLE = True
except Exception:
    HAVE_PADDLE = try_install_paddle()
    try:
        if HAVE_PADDLE:
            from paddleocr import PaddleOCR  # type: ignore
        else:
            raise ImportError
    except Exception:
        HAVE_PADDLE = False

if HAVE_PADDLE:
    run_paddle_ocr(image_paths)
else:
    # Create a placeholder so downstream steps don’t break
    print("PaddleOCR not available in this runtime. Proceeding with Tesseract + EasyOCR only.")
    os.makedirs("outputs/paddleocr", exist_ok=True)
    with open("outputs/paddleocr/all_text.txt","w",encoding="utf-8") as f:
        f.write("(PaddleOCR not available in this Colab runtime)")


Skipping PaddleOCR install: Python 3.11.13 is not eligible (>= 3.11).
PaddleOCR not available in this runtime. Proceeding with Tesseract + EasyOCR only.


In [14]:
def head_text(p, n=600):
    import os
    return open(p, "r", encoding="utf-8").read()[:n].replace("\n"," ") if os.path.exists(p) else "(missing)"

print("=== Tesseract (head) ==="); print(head_text("tesseract_output.txt") or head_text("outputs/tesseract/all_text.txt"))
print("\n=== EasyOCR (head) ==="); print(head_text("easyocr_output.txt") or head_text("outputs/easyocr/all_text.txt"))
print("\n=== PaddleOCR (head) ==="); print(head_text("paddleocr_output.txt") or head_text("outputs/paddleocr/all_text.txt"))

import pandas as pd, os
def contains_any(text, needles):
    tl = text.lower()
    return any(n.lower() in tl for n in needles)

eng = {
    "tesseract": "tesseract_output.txt" if os.path.exists("tesseract_output.txt") else "outputs/tesseract/all_text.txt",
    "easyocr": "easyocr_output.txt" if os.path.exists("easyocr_output.txt") else "outputs/easyocr/all_text.txt",
    "paddleocr": "paddleocr_output.txt" if os.path.exists("paddleocr_output.txt") else "outputs/paddleocr/all_text.txt",
}
needles = {
    "loan_amount": ["loan amount", "$380,000", "380,000"],
    "lender": ["xyz lender", "lender:"],
    "interest": ["4.250 %","4.250%","interest rate"],
    "program": ["30 year fixed","30-year fixed","30 yr fixed"],
    "applicants": ["john q. smith","mary a. smith"],
}
rows=[]
for name, path in eng.items():
    txt = open(path, "r", encoding="utf-8").read() if os.path.exists(path) else ""
    row={"engine": name}
    for k, ns in needles.items():
        row[k] = contains_any(txt, ns)
    rows.append(row)

df = pd.DataFrame(rows); display(df)
df.to_csv("key_field_presence.csv", index=False)
print("Saved key_field_presence.csv")


=== Tesseract (head) ===
Your actual rate, payment, and cost could be higher. Get an official Loan Estimate before choosing a loan.  FEES WORKSHEET Fee Details and Summary  Applicants: John Q. Smith / Mary A. Smith Application No: samplesmith Prepared By: XYZ Lender Date Prepared: 10/05/2015 Loan Program: 30 YEAR FIXED -Purchase  THIS IS NOT A GOOD FAITH ESTIMATE (GFE). This "Fees Worksheet" is provided for informational purposes ONLY, to assist you in determining an estimate of cash that may be required to close and an estimate of your proposed monthly mortgage payment. Actual charges may be more or less, and your tr

=== EasyOCR (head) ===
Your actual rate, payment; and cost could be higher. Get an official Loan Estimate before choosing a Ioan. FEES WORKSHEET Fee Details and Summary Applicants: John Q. Smith A. Smith Application No: samplesmith Prepared By: XYZ Lender Date Prepared: 10/05/2015 Loan Program: 30 YEAR FIXED -Purchase THIS IS NOT A GOOD FAITH ESTIMATE (GFE): This "Fees W

,engine,loan_amount,lender,interest,program,applicants
0,tesseract,True,True,True,True,True
1,easyocr,True,True,True,True,True
2,paddleocr,False,False,False,False,False


Saved key_field_presence.csv


In [15]:
import shutil, datetime, os
stamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
bundle = f"OCR_Task2_Outputs_{stamp}"
os.makedirs(bundle, exist_ok=True)

# Include text outputs, boxed images (if you generated them), pages, and CSV
for p in [
    "pages",
    "outputs",                   # from the earlier step-by-step
    "tesseract_output.txt",
    "easyocr_output.txt",
    "paddleocr_output.txt",
    "key_field_presence.csv",
]:
    if os.path.isdir(p):
        shutil.copytree(p, os.path.join(bundle, os.path.basename(p)), dirs_exist_ok=True)
    elif os.path.exists(p):
        shutil.copy2(p, os.path.join(bundle, os.path.basename(p)))

shutil.make_archive(bundle, "zip", bundle)
print("Created ZIP:", bundle + ".zip")


Created ZIP: OCR_Task2_Outputs_20250810_0351.zip


In [16]:
# --- Reflection ---
from IPython.display import Markdown

reflection_text = """
## Reflection
Using PyMuPDF, I extracted structured text from the Loan Worksheet PDF with high accuracy for most content. The tool effectively captured body text, headers, and even multi-line fields in logical reading order. However, some alignment-dependent sections such as fee tables and multi-column layouts caused minor text grouping issues. In certain cases, values from adjacent columns appeared concatenated or slightly misaligned, which could affect downstream parsing. Despite these minor challenges, PyMuPDF handled the majority of the text consistently and preserved positional data, which is valuable for refining field extraction rules. Compared to scanned or low-quality documents, this text-based PDF extraction was faster and cleaner, avoiding OCR-related noise. Overall, the process confirmed that PyMuPDF is highly reliable for digital PDFs, but table-structured or multi-column data may require additional logic to segment fields precisely.
"""

key_insights_text = """
## Key Insights
In the next phase, extracting fields like Loan Amount and Interest Rate will require combining positional data with keyword anchors. Since some values appear close to similar terms or in tabular formats, adding bounding-box filtering will help isolate the correct numbers. For example, detecting “Loan Amount” and capturing the nearest numeric value within a defined horizontal/vertical range would reduce errors from unrelated amounts. Additionally, handling multi-column layouts separately from single-column text will avoid mis-grouping. Another improvement would be applying light preprocessing to normalize spacing and punctuation before regex searches. These adjustments will make the pipeline more accurate and consistent, especially when extracting critical numeric fields from varied mortgage document formats.
"""

display(Markdown(reflection_text))
display(Markdown(key_insights_text))



## Reflection
Using PyMuPDF, I extracted structured text from the Loan Worksheet PDF with high accuracy for most content. The tool effectively captured body text, headers, and even multi-line fields in logical reading order. However, some alignment-dependent sections such as fee tables and multi-column layouts caused minor text grouping issues. In certain cases, values from adjacent columns appeared concatenated or slightly misaligned, which could affect downstream parsing. Despite these minor challenges, PyMuPDF handled the majority of the text consistently and preserved positional data, which is valuable for refining field extraction rules. Compared to scanned or low-quality documents, this text-based PDF extraction was faster and cleaner, avoiding OCR-related noise. Overall, the process confirmed that PyMuPDF is highly reliable for digital PDFs, but table-structured or multi-column data may require additional logic to segment fields precisely.



## Key Insights
In the next phase, extracting fields like Loan Amount and Interest Rate will require combining positional data with keyword anchors. Since some values appear close to similar terms or in tabular formats, adding bounding-box filtering will help isolate the correct numbers. For example, detecting “Loan Amount” and capturing the nearest numeric value within a defined horizontal/vertical range would reduce errors from unrelated amounts. Additionally, handling multi-column layouts separately from single-column text will avoid mis-grouping. Another improvement would be applying light preprocessing to normalize spacing and punctuation before regex searches. These adjustments will make the pipeline more accurate and consistent, especially when extracting critical numeric fields from varied mortgage document formats.
